# Homework 8
Currently breaking it up into subsections that I can work on

QUESTIONS:

Should you be able to upload database files also??

# Part 1: Basic Flask Website Architecture

In [51]:
%%writefile templates/base.html
<!DOCTYPE html>
<html>
    <head>
        <title>My Super Site</title>
    </head>

    <body>
        <h1>{{ page_title }}</h1>
        
        <h2>
    <a href="{{ url_for('home') }}">Home</a>
    <a href="{{ url_for('query_db') }}">Query Database</a>
    <a href="{{ url_for('upload_file') }}">Upload a File</a>
    </h2>

        
{% block content %}{% endblock %}        
    </body>
</html>


Overwriting templates/base.html


In [52]:
%%writefile templates/form.html
{% extends "base.html" %}
{% set page_title = 'My Form' %}
{% block content %}
            <form action="/query_database/" method="POST">
            Make a query: 
            <input type="text" name="query" />
            <input type="submit" />
            </form>
{% endblock %}

Overwriting templates/form.html


In [24]:
%%writefile templates/file_up.html
{% extends "base.html" %}

{% set page_title = 'Choose file to upload' %}
{% block content %}
   
      <form action = "/upload_file/" method = "POST" 
         enctype = "multipart/form-data">
        <input type = "text" name = "collection_name" />

         <input type = "file" name = "file" />
         <input type = "submit"/>
      </form>
      
{% endblock %}

Overwriting templates/file_up.html


In [27]:
%%writefile basic_site.py

from flask import Flask, render_template, request, url_for, redirect
import flask
import pybtex.database as pb

app = Flask(__name__)
app.debug = True

@app.route("/")
@app.route("/home")
def home():
    return render_template('base.html', 
                            page_title="Home", 
                            content="woo")

#No idea why I need this, but I do :()
@app.route("/login")
def redirect_to_home():
    y+=1
    print('and finally here')
    ## 301 is an HTTP error code
    return redirect("/home",301)


@app.route('/query_database/', methods=['GET', 'POST'])
def query_db():
    if request.method == 'POST':
        query = request.form['query']
        if query not in (""," ",None):
            return render_template('base.html', 
                                   page_title= query, 
                                   content="You queried %s. Too bad nothing works yet." % query)
        else:
            return render_template('base.html', 
                                   page_title= 'Nope', 
                                   content='Thats so not a query. come on man.' +
                                   'Please give a query here: BUT MY LINKS ARENT HERE YET')
    else:
        ## this is a normal GET request
        return render_template("form.html", page_title = 'Query a Database :)')
    
    
    
@app.route('/upload_file/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:

            return render_template('base.html', 
                                   page_title= "NO FILE?!??", content ='boo')
        file = request.files['file']
        if file not in (""," ",None):
            coll_name = request.form['collection_name']
            if coll_name not in (""," ",None):
                #read and convert to this database thing.
                x=pb.parse_bytes(file.read(), bib_format='bibtex')
                return render_template('base.html', 
                                       page_title= coll_name,
                                       content="nice")
        else:
            return render_template('base.html', 
                                   page_title= 'Nope', 
                                   content='Thats so not a query. come on man.' +
                                   'Please give a query here: BUT MY LINKS ARENT HERE YET')
    else:
        ## this is a normal GET request
        return render_template("file_up.html", page_title = 'Upload a File :)')


if __name__ == "__main__":
    app.run()

Overwriting basic_site.py


In [58]:
%run basic_site.py

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


# Part 2: Add travis testing to Basic Flask

# Part 3: Build a Database

# Part 4: Add travis to database

# Part 5: Combine database + Flask

# Part 6: add travis to that.